In [ ]:
import os
os.environ['SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL'] = 'True'
import subprocess
subprocess.run(['pip', 'install', 'sklearn'])

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten
import pandas as pd
import numpy as np
import fastparquet as fp
import time
from sklearn.metrics import r2_score, root_mean_squared_error
import matplotlib.pyplot as plt
import jax
import tensorflow as tf
keras.utils.set_random_seed(1)

ImportError: cannot import name 'clip_to_image_size' from 'keras.src.layers.preprocessing.image_preprocessing.bounding_boxes.converters' (C:\Users\arsen\anaconda3\envs\keras_jax\lib\site-packages\keras\src\layers\preprocessing\image_preprocessing\bounding_boxes\converters.py)

In [ ]:
from sigtkan_manual import SigTKAN
from tkan import TKAN

In [ ]:
df = pd.read_parquet('data.parquet')
df = df[(df.index >= pd.Timestamp('2020-01-01')) & (df.index < pd.Timestamp('2023-01-01'))]
assets = ['BTC', 'ETH', 'ADA', 'XMR', 'EOS', 'MATIC', 'TRX', 'FTM', 'BNB', 'XLM', 'ENJ', 'CHZ', 'BUSD', 'ATOM', 'LINK', 'ETC', 'XRP', 'BCH', 'LTC']
df = df[[c for c in df.columns if 'quote asset volume' in c and any(asset in c for asset in assets)]]
df.columns = [c.replace(' quote asset volume', '') for c in df.columns]
df = df.head(500)
display(df)

In [ ]:
class MinMaxScaler:
    def __init__(self, feature_axis=None, minmax_range=(0, 1)):
        """
        Initialize the MinMaxScaler.
        Args:
        feature_axis (int, optional): The axis that represents the feature dimension if applicable.
                                      Use only for 3D data to specify which axis is the feature axis.
                                      Default is None, automatically managed based on data dimensions.
        """
        self.feature_axis = feature_axis
        self.min_ = None
        self.max_ = None
        self.scale_ = None
        self.minmax_range = minmax_range # Default range for scaling (min, max)

    def fit(self, X):
        """
        Fit the scaler to the data based on its dimensionality.
        Args:
        X (np.array): The data to fit the scaler on.
        """
        if X.ndim == 3 and self.feature_axis is not None:  # 3D data
            axis = tuple(i for i in range(X.ndim) if i != self.feature_axis)
            self.min_ = np.min(X, axis=axis)
            self.max_ = np.max(X, axis=axis)
        elif X.ndim == 2:  # 2D data
            self.min_ = np.min(X, axis=0)
            self.max_ = np.max(X, axis=0)
        elif X.ndim == 1:  # 1D data
            self.min_ = np.min(X)
            self.max_ = np.max(X)
        else:
            raise ValueError("Data must be 1D, 2D, or 3D.")

        self.scale_ = self.max_ - self.min_
        return self

    def transform(self, X):
        """
        Transform the data using the fitted scaler.
        Args:
        X (np.array): The data to transform.
        Returns:
        np.array: The scaled data.
        """
        X_scaled = (X - self.min_) / self.scale_
        X_scaled = X_scaled * (self.minmax_range[1] - self.minmax_range[0]) + self.minmax_range[0]
        return X_scaled

    def fit_transform(self, X):
        """
        Fit to data, then transform it.
        Args:
        X (np.array): The data to fit and transform.
        Returns:
        np.array: The scaled data.
        """
        return self.fit(X).transform(X)

    def inverse_transform(self, X_scaled):
        """
        Inverse transform the scaled data to original data.
        Args:
        X_scaled (np.array): The scaled data to inverse transform.
        Returns:
        np.array: The original data scale.
        """
        X = (X_scaled - self.minmax_range[0]) / (self.minmax_range[1] - self.minmax_range[0])
        X = X * self.scale_ + self.min_
        return X

def generate_data(df, sequence_length, n_ahead = 1):
    #Case without known inputs
    scaler_df = df.copy().shift(n_ahead).rolling(24 * 14).median()
    tmp_df = df.copy() / scaler_df
    tmp_df = tmp_df.iloc[24 * 14 + n_ahead:].fillna(0.)
    scaler_df = scaler_df.iloc[24 * 14 + n_ahead:].fillna(0.)
    def prepare_sequences(df, scaler_df, n_history, n_future):
        X, y, y_scaler = [], [], []
        num_features = df.shape[1]
        
        # Iterate through the DataFrame to create sequences
        for i in range(n_history, len(df) - n_future + 1):
            # Extract the sequence of past observations
            X.append(df.iloc[i - n_history:i].values)
            # Extract the future values of the first column
            y.append(df.iloc[i:i + n_future,0:1].values)
            y_scaler.append(scaler_df.iloc[i:i + n_future,0:1].values)
        
        X, y, y_scaler = np.array(X), np.array(y), np.array(y_scaler)
        return X, y, y_scaler
    
    # Prepare sequences
    X, y, y_scaler = prepare_sequences(tmp_df, scaler_df, sequence_length, n_ahead)
    
    # Split the dataset into training and testing sets
    train_test_separation = int(len(X) * 0.8)
    X_train_unscaled, X_test_unscaled = X[:train_test_separation], X[train_test_separation:]
    y_train_unscaled, y_test_unscaled = y[:train_test_separation], y[train_test_separation:]
    y_scaler_train, y_scaler_test = y_scaler[:train_test_separation], y_scaler[train_test_separation:]
    
    # Generate the data
    X_scaler = MinMaxScaler(feature_axis=2)
    X_train = X_scaler.fit_transform(X_train_unscaled)
    X_test = X_scaler.transform(X_test_unscaled)
    
    y_scaler = MinMaxScaler(feature_axis=2)
    y_train = y_scaler.fit_transform(y_train_unscaled)
    y_test = y_scaler.transform(y_test_unscaled)
    
    y_train = y_train.reshape(y_train.shape[0], -1) 
    y_test = y_test.reshape(y_test.shape[0], -1)
    return X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test

In [ ]:
early_stopping_callback = lambda : tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,    
    patience=6,         
    mode="min",
    restore_best_weights=True,
    start_from_epoch=3, 
    verbose=1,
)

lr_callback = lambda : tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=3,
    mode="min",
    min_delta=0.0001,  
    min_lr=0.000025,
    verbose=1,
)

callbacks = lambda : [early_stopping_callback(), lr_callback(), tf.keras.callbacks.TerminateOnNaN()]

In [ ]:
def tkan_config(return_sequences):
    return {
        "units": 100,
        "dropout": 0.0,
        "sub_kan_output_dim": 20,
        "sub_kan_input_dim": 20,
        "return_sequences": return_sequences
    }

n_aheads = [1]
models = ["TKAN", "SigTKAN"]
N_MAX_EPOCHS = 1000
BATCH_SIZE = 128

callbacks = lambda : [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, verbose=1),
    tf.keras.callbacks.TerminateOnNaN()
]

results = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
results_rmse = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
time_results = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
all_errors = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}

for n_ahead in n_aheads:
    sequence_length = max(45, 5 * n_ahead)
    X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test = generate_data(df, sequence_length, n_ahead)
    
    for model_id in models:
        for run in range(10):
            if model_id == 'TKAN':
                model = Sequential([
                    Input(shape=X_train.shape[1:]),
                    TKAN(**tkan_config(return_sequences=True)),
                    TKAN(**tkan_config(return_sequences=False)),
                    Dense(units=n_ahead, activation='linear')
                ], name=model_id)

            elif model_id == 'SigTKAN':
                model = Sequential([
                    Input(shape=X_train.shape[1:]),
                    SigTKAN(sig_level=2, **tkan_config(return_sequences=True)),
                    SigTKAN(sig_level=2, **tkan_config(return_sequences=False)),
                    Dense(units=n_ahead, activation='linear')
                ], name=model_id)

            else:
                raise ValueError(f"Unknown model_id: {model_id}")

            optimizer = tf.keras.optimizers.Adam(0.0001)
            model.compile(optimizer=optimizer, loss='mean_squared_error', jit_compile=True)
            if run == 0:
                model.summary()

            start_time = time.time()
            history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=N_MAX_EPOCHS, validation_split=0.2, callbacks=callbacks(), shuffle=True, verbose=False)
            end_time = time.time()
            time_results[model_id][n_ahead].append(end_time - start_time)

            preds = model.predict(X_test, verbose=False)
            r2 = r2_score(y_true=y_test, y_pred=preds)
            rmse = np.sqrt(np.mean((y_test - preds) ** 2))
            
            errors = preds - y_test
            all_errors[model_id][n_ahead].append(errors)

            print(end_time - start_time, r2)

            results[model_id][n_ahead].append(r2)
            results_rmse[model_id][n_ahead].append(rmse)

            del model
            del optimizer


print('R2 scores - Means:')
display(pd.DataFrame({model_id: {n_ahead: np.mean(results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results}))

print('RMSE scores - Means:')
display(pd.DataFrame({model_id: {n_ahead: np.mean(results_rmse[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results_rmse}))

print('R2 scores - Std:')
display(pd.DataFrame({model_id: {n_ahead: np.std(results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results}))

print('RMSE scores - Std:')
display(pd.DataFrame({model_id: {n_ahead: np.std(results_rmse[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results_rmse}))

print('Training Times - Means:')
display(pd.DataFrame({model_id: {n_ahead: np.mean(time_results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in time_results}))

print('Training Times - Std:')
display(pd.DataFrame({model_id: {n_ahead: np.std(time_results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in time_results}))

In [ ]:
# Préparer all_errors pour le plot
all_errors_plot = {}
all_errors_plot['TKAN'] = np.mean(all_errors['TKAN'][1], axis=0)
all_errors_plot['SigTKAN'] = np.mean(all_errors['SigTKAN'][1], axis=0)    

model_types = ['TKAN', 'SigTKAN']
colors = ['#1f77b4', '#2ca02c'] 

for model_type, color in zip(model_types, colors):
    y_pred = all_errors_plot[model_type] + y_test
    r2 = r2_score(y_true=y_test.flatten(), y_pred=y_pred.flatten())
    plt.plot(np.mean(all_errors_plot[model_type]**2, axis=0), label=f'{model_type}: R²={round(r2,4)}', color=color)

plt.legend()
plt.title('Model and errors based on number of steps forward')
plt.xlabel('number of steps forward')
plt.ylabel('Mean Squared Error')
plt.show()